In [ ]:
import numpy as np
import pandas as pd
import datetime

# df = pd.read_csv ('CGMData.csv')
# print(df)



CGM_data_extractor = pd.read_csv('CGMData.csv')
CGM_data_extractor.rename({'Sensor Glucose (mg/dL)': 'Gluco-(mg/dL)'}, axis=1, inplace=True)
 

Insulin_data_extractor = pd.read_csv('InsulinData.csv')

CGM_data_extractor['datetime'] = pd.to_datetime(CGM_data_extractor['Date']+" "+CGM_data_extractor['Time'])
Insulin_data_extractor['datetime'] = pd.to_datetime(Insulin_data_extractor['Date']+" "+Insulin_data_extractor['Time'])


auto_mode_info = Insulin_data_extractor[Insulin_data_extractor['Alarm']=='AUTO MODE ACTIVE PLGM OFF']


auto_mode_time = auto_mode_info.iloc[-1]['datetime']

# auto_mode_time = auto_mode_info.iloc[-1]['Time']

# auto_mode_time = pd.to_datetime(str(auto_mode_da) +" "+str(auto_mode_tim))


# print(CGM_data_extractor)
# print(Insulin_data_extractor)
print(auto_mode_time)


CGM_auto_mode = CGM_data_extractor[CGM_data_extractor["datetime"]>=auto_mode_time]
CGM_manual_mode = CGM_data_extractor[CGM_data_extractor["datetime"]<auto_mode_time]

CGM_auto_mode.loc["Date"] = pd.to_datetime(CGM_auto_mode["Date"])
CGM_auto_mode.loc["Time"] = pd.to_datetime(CGM_auto_mode["Time"])

CGM_manual_mode.loc["Date"] = pd.to_datetime(CGM_manual_mode["Date"])
CGM_manual_mode.loc["Time"] = pd.to_datetime(CGM_manual_mode["Time"])


# print(CGM_auto_mode, CGM_manual_mode )



CGM_auto_daytime = CGM_auto_mode[CGM_auto_mode['datetime'].dt.hour>=6].copy()
CGM_auto_overnight = CGM_auto_mode[CGM_auto_mode['datetime'].dt.hour<6].copy()
CGM_manual_daytime = CGM_manual_mode[CGM_manual_mode['datetime'].dt.hour>=6].copy()
CGM_manual_overnight = CGM_manual_mode[CGM_manual_mode['datetime'].dt.hour<6].copy()




def mean(dataset, days):
    total = dataset.groupby(by='Date')['Time'].count()
    if(len(total) == 0):
      return 0.0 
    summ = (total/288)*100
    return (summ.sum()/days)



manual = [CGM_manual_overnight, CGM_manual_daytime, CGM_manual_mode]
auto = [CGM_auto_overnight, CGM_auto_daytime,  CGM_auto_mode]
results =[[], []]




def metricextractor(dataset, flag):
  if flag == 'manual':
    j = 0
  else:
    j = 1
  for evaluating_data in dataset:
      days = evaluating_data['Date'].nunique()
      results[j].append(mean(evaluating_data[evaluating_data['Gluco-(mg/dL)']>180], days))
      results[j].append(mean(evaluating_data[evaluating_data['Gluco-(mg/dL)']>250], days))
      results[j].append(mean(evaluating_data[(evaluating_data['Gluco-(mg/dL)']>=70) & (evaluating_data['Gluco-(mg/dL)']<=180)], days))
      results[j].append(mean(evaluating_data[(evaluating_data['Gluco-(mg/dL)']>=70) & (evaluating_data['Gluco-(mg/dL)']<=150)], days))
      results[j].append(mean(evaluating_data[evaluating_data['Gluco-(mg/dL)']<70], days))
      results[j].append(mean(evaluating_data[evaluating_data['Gluco-(mg/dL)']<54], days))
  results[j].append(1.1)



metricextractor(auto, "auto")
metricextractor(manual, "manual")

df = pd.DataFrame(results)
print(results)

df
df.to_csv('Results.csv',index=False, header= False)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (32) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (13,43) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


2017-08-09 08:07:13


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


[[4.837962962962963, 1.0185185185185186, 18.240740740740744, 13.402777777777775, 0.6712962962962963, 0.0, 22.829861111111107, 8.463541666666666, 34.830729166666664, 27.105034722222225, 4.578993055555555, 1.8012152777777777, 27.365451388888886, 9.418402777777777, 51.93142361111111, 39.670138888888886, 5.208333333333333, 1.8012152777777777, 1.1], [2.600265293383271, 0.4057428214731585, 20.265683520599254, 17.68102372034956, 0.6300717852684145, 0.15215355805243444, 19.535108024691358, 4.560185185185185, 45.11574074074075, 33.80015432098765, 3.31983024691358, 0.9837962962962962, 22.10648148148148, 4.961419753086419, 65.15625, 51.28472222222222, 3.942901234567901, 1.1342592592592593, 1.1]]


# New Section